In [ ]:
!pip install pdfplumber tiktoken openai chromaDB sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
 #Define the path of the PDF
csv_path = '/content/drive/MyDrive/Colab Notebooks/updated_car.csv'

In [ ]:
# Set the API key
filepath = "/content/drive/MyDrive/Colab Notebooks/"

with open(filepath + "api_key_3.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [ ]:
# Define the path where chroma collections will be stored

chroma_data_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDB_Data'

In [ ]:
import chromadb

In [ ]:
# Call PersistentClient()

client = chromadb.PersistentClient()

In [ ]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [ ]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents

cars_collection = client.get_or_create_collection(name='RAG_on_UsedCars', embedding_function=embedding_function)

In [ ]:
cardf = pd.read_csv(csv_path,nrows=300)
cardf['car_details']=cardf['Car Name'].astype(str) + ' ' + cardf['Make'].astype(str)+ ' ' + cardf['Model'].astype(str)+ ' ' + cardf['Year'].astype(str)+ ' ' + cardf['Price'].astype(str)+ ' ' + cardf['Fuel'].astype(str)+ ' ' + cardf['Transmission'].astype(str)+ ' ' + cardf['Car Features'].astype(str)+ cardf["KM's driven"].astype(str)
# columns_to_keep = ['car_details']  # Specify the column names you want to keep
# newcardf = pd.read_csv(csv_path, usecols=columns_to_keep)
cardf.to_csv(filepath+'mergedcars.csv', index=False)
# cardf

In [ ]:
chunk_size = 10000  # You can adjust this based on your system's memory constraints

# Initialize an empty list to store the chunks
chunks = []

# Iterate over the CSV file in chunks
for chunk in pd.read_csv(filepath+'mergedcars.csv', chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate the chunks into a single DataFrame
chunkedcardf = pd.concat(chunks, ignore_index=True)

In [ ]:
chunkedcardf

,Ad ID,Car Name,Make,Model,Year,KM's driven,Price,Fuel,Registration city,Car documents,Assembly,Transmission,Condition,Seller Location,Description,Car Features,Images URL's,Car Profile,car_extracted_features,car_details
0,1079071571,fresh import Passo 2021model,Toyota,Passo,2021,54000,4190000,Petrol,Unregistered,Original,Imported,Automatic,Used,"Airline Avenue, Islamabad","it's 2021 model fresh import, perfect engine s...","ABS, Air Bags, AM/FM Radio, CD Player, Cassett...",['https://images.olx.com.pk/thumbnails/4039460...,https://www.olx.com.pk/item/fresh-import-passo...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",fresh import Passo 2021model Toyota Passo 2021...
1,1080125520,Suzuki ravi,Suzuki,Ravi,2018,95000,1300000,Petrol,Karachi,Original,Local,Manual,Used,"Kahuta, Rawalpindi",Suzuki ravi 2018 col,AM/FM Radio,['https://images.olx.com.pk/thumbnails/4102504...,https://www.olx.com.pk/item/suzuki-ravi-iid-10...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Suzuki ravi Suzuki Ravi 2018 1300000 Petrol Ma...
2,1080748789,Suzuki bolan 2015 contact 03112271054,Suzuki,Bolan,2015,50000,800000,Petrol,Karachi,Original,Local,Manual,Used,"Lyari Expressway, Karachi",Suzuki bolan model 2015 reg 2022 ghadi me koi ...,Rear speakers,['https://images.olx.com.pk/thumbnails/4139520...,https://www.olx.com.pk/item/suzuki-bolan-2015-...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Suzuki bolan 2015 contact 03112271054 Suzuki B...
3,1076081635,Diahatsu Move 2013,Daihatsu,Move,2013,94000,2155000,Petrol,Lahore,Original,Imported,Automatic,Used,"New Amir Town, Lahore",Move Push start \nHome Used car. \nModel 2013....,"ABS, Air Bags, Air Conditioning, Alloy Rims, A...",['https://images.olx.com.pk/thumbnails/3865337...,https://www.olx.com.pk/item/diahatsu-move-2013...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Diahatsu Move 2013 Daihatsu Move 2013 2155000 ...
4,1080812928,Suzuki Swift DLX 2011 miner tuchap,Suzuki,Swift,2011,126544,1440000,Petrol,Karachi,Original,Local,Manual,Used,"Shadman 2, Karachi",Suzuki Swift DLX\nAbS Break 100%\nengine 100%\...,"ABS, Air Conditioning, Alloy Rims, AM/FM Radio...",['https://images.olx.com.pk/thumbnails/4143509...,https://www.olx.com.pk/item/suzuki-swift-dlx-2...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Suzuki Swift DLX 2011 miner tuchap Suzuki Swif...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1079468164,FAW xpv for sale,FAW,X-PV,2013,183000,600000,Petrol,Chiniot,Original,Imported,Manual,Used,"Chinioti Road, Chiniot","FAW xpv 2013 model for sale non Ac,Ac mojod nh...","Front Camera, Keyless Entry, Power Steering, R...",['https://images.olx.com.pk/thumbnails/4069218...,https://www.olx.com.pk/item/faw-xpv-for-sale-i...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",FAW xpv for sale FAW X-PV 2013 600000 Petrol M...
296,1070931590,Toyota Corolla GLI 2010 Model For Sale,Toyota,Corolla GLI,2010,10880,2375000,Petrol,Lahore,Original,Local,Manual,Used,"Ichhra, Lahore",Toyota Corolla GLI \n2010 Model For Sale\n Nic...,"Alloy Rims, AM/FM Radio, DVD Player, Power Loc...",['https://images.olx.com.pk/thumbnails/3914360...,https://www.olx.com.pk/item/toyota-corolla-gli...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Toyota Corolla GLI 2010 Model For Sale Toyota ...
297,1080830526,Honda city 2011,Honda,City IVTEC,2011,225000,2300000,Petrol,Karachi,Original,Imported,Manual,Used,"G-7, Islamabad",car is in good condition. \n033 six 529 six 58...,"Air Conditioning, Alloy Rims, AM/FM Radio, Coo...",['https://images.olx.com.pk/thumbnails/4144515...,https://www.olx.com.pk/item/honda-city-2011-ii...,"\n{\n 'Km driven': 'medium',\n 'Fuel typ...",Honda city 2011 Honda City IVTEC 2011 2300000 ...
298,1080647828,alloy Ram original colour chil AC,Daihatsu,Mira,2007,200000,750000,Petrol,Karachi,Original,Local,Manual,Used,"Chak 8 DNB, Bahawalpur",Daihatsu liar 2007 \n2008 registration\norigin...,Air Conditioning,['https://images.olx.com.pk/thumbnails/4133550...,https://www.olx.com.pk/item/alloy

In [ ]:
# Store the metadata for each page in a separate column

chunkedcardf['Metadata'] = chunkedcardf.apply(lambda x: {'Make': x['Make'], 'Car Name.': x['Car Name']}, axis=1)

In [ ]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma

cars_list = chunkedcardf["car_details"].tolist()
metadata_list = chunkedcardf['Metadata'].tolist()

In [ ]:
cars_collection.add(
    documents= cars_list,
    ids = [str(i) for i in range(0, len(cars_list))],
    metadatas = metadata_list
)

In [ ]:
# Let's take a look at the first few entries in the collection

cars_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': [[-0.01665317267179489,
   -0.008632327429950237,
   0.0009131597471423447,
   -0.025851447135210037,
   -0.02386087365448475,
   0.01643199659883976,
   0.006131099537014961,
   -0.01990574412047863,
   -0.01651005819439888,
   -0.01855267398059368,
   0.022091472521424294,
   0.01149459183216095,
   -0.02976754494011402,
   -0.015365153551101685,
   -0.013478660956025124,
   0.006732825189828873,
   0.0032102891709655523,
   -0.017355727031826973,
   0.0003323722630739212,
   -0.005880651529878378,
   -0.011071757413446903,
   -0.0011652341345325112,
   -0.005454564467072487,
   -0.009679656475782394,
   -0.013361568562686443,
   0.02510986104607582,
   0.01714756339788437,
   -0.024459345266222954,
   0.019138136878609657,
   -0.03002775087952614,
   0.007604514714330435,
   -0.015078926458954811,
   -0.035908401012420654,
   -0.022885100916028023,
   0.0007184119895100594,
   -0.03947322070598602,
   -0.0052886828780174255,
   -0.021506009623

In [ ]:
cache_collection = client.get_or_create_collection(name='Cars_Cache', embedding_function=embedding_function)

In [ ]:
cache_collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [ ]:
query = input()

Tell me the petrol cars which have automatic transmission and manufactured after 2001


In [ ]:
# Searh the Cache collection first
# Query the collection against the user query and return the top 20 results

cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [ ]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = cars_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      for key, val in results.items():
        if val is None:
          continue
        for i in range(10):
          Keys.append(str(key)+str(i))
          Values.append(str(val[0][i]))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

Not found in cache. Found in main collection.


In [ ]:
results_df

,Metadatas,Documents,Distances,IDs
0,{'Car Name.': 'Toyota Altis automatic for sale...,Toyota Altis automatic for sale Toyota Corroll...,0.334184,69
1,"{'Car Name.': 'Toyota Altis 1.6', 'Make': 'Toy...",Toyota Altis 1.6 Toyota Corrolla Altis 2018 47...,0.336700,288
2,"{'Car Name.': 'Honda civic 2004 automatic', 'M...",Honda civic 2004 automatic Honda Civic Oriel 2...,0.341661,255
3,"{'Car Name.': 'HONDA CITY 2001 Auto', 'Make': ...",HONDA CITY 2001 Auto Honda City IDSI 2001 9500...,0.345812,65
4,"{'Car Name.': 'Corolla Altis 1.6 automatic', '...",Corolla Altis 1.6 automatic Toyota Corrolla Al...,0.348494,262
5,"{'Car Name.': 'Honda city 1.2L CVT', 'Make': '...",Honda city 1.2L CVT Honda City IVTEC 2022 4600...,0.352806,167
6,"{'Car Name.': 'Xli 2020 model Saaf gari', 'Mak...",Xli 2020 model Saaf gari Toyota Corolla XLI 20...,0.355114,83
7,"{'Car Name.': 'Honda city 2011', 'Make': 'Honda'}",Honda city 2011 Honda City IVTEC 2011 2300000 ...,0.357912,297
8,"{'Car Name.': 'Daihatsu Coure 2011 Automatic',...",Daihatsu Coure 2011 Automatic Daihatsu Cuore 2...,0.359949,78
9,"{'Car Name.': 'Toyota Corolla 1.3 GLi Manual',...",Toyota Corolla 1.3 GLi Manual Toyota Corolla G...,0.362789,92


In [ ]:
from sentence_transformers import CrossEncoder, util

In [ ]:
# Initialise the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [ ]:
cross_rerank_scores

array([-4.434132 , -5.0914497, -4.7591796, -1.1150271, -4.6271358,
       -6.6651354, -5.9426093, -9.910283 , -7.3116307, -8.200621 ],
      dtype=float32)

In [ ]:
# Store the rerank_scores in results_df

results_df['Reranked_scores'] = cross_rerank_scores

In [ ]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Car Name.': 'Toyota Altis automatic for sale...,Toyota Altis automatic for sale Toyota Corroll...,0.334184,69,-4.434132
1,"{'Car Name.': 'Toyota Altis 1.6', 'Make': 'Toy...",Toyota Altis 1.6 Toyota Corrolla Altis 2018 47...,0.336700,288,-5.091450
2,"{'Car Name.': 'Honda civic 2004 automatic', 'M...",Honda civic 2004 automatic Honda Civic Oriel 2...,0.341661,255,-4.759180
3,"{'Car Name.': 'HONDA CITY 2001 Auto', 'Make': ...",HONDA CITY 2001 Auto Honda City IDSI 2001 9500...,0.345812,65,-1.115027
4,"{'Car Name.': 'Corolla Altis 1.6 automatic', '...",Corolla Altis 1.6 automatic Toyota Corrolla Al...,0.348494,262,-4.627136
5,"{'Car Name.': 'Honda city 1.2L CVT', 'Make': '...",Honda city 1.2L CVT Honda City IVTEC 2022 4600...,0.352806,167,-6.665135
6,"{'Car Name.': 'Xli 2020 model Saaf gari', 'Mak...",Xli 2020 model Saaf gari Toyota Corolla XLI 20...,0.355114,83,-5.942609
7,"{'Car Name.': 'Honda city 2011', 'Make': 'Honda'}",Honda city 2011 Honda City IVTEC 2011 2300000 ...,0.357912,297,-9.910283
8,"{'Car Name.': 'Daihatsu Coure 2011 Automatic',...",Daihatsu Coure 2011 Automatic Daihatsu Cuore 2...,0.359949,78,-7.311631
9,"{'Car Name.': 'Toyota Corolla 1.3 GLi Manual',...",Toyota Corolla 1.3 GLi Manual Toyota Corolla G...,0.362789,92,-8.200621


In [ ]:
top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Car Name.': 'Toyota Altis automatic for sale...,Toyota Altis automatic for sale Toyota Corroll...,0.334184,69,-4.434132
1,"{'Car Name.': 'Toyota Altis 1.6', 'Make': 'Toy...",Toyota Altis 1.6 Toyota Corrolla Altis 2018 47...,0.336700,288,-5.091450
2,"{'Car Name.': 'Honda civic 2004 automatic', 'M...",Honda civic 2004 automatic Honda Civic Oriel 2...,0.341661,255,-4.759180


In [ ]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
3,"{'Car Name.': 'HONDA CITY 2001 Auto', 'Make': ...",HONDA CITY 2001 Auto Honda City IDSI 2001 9500...,0.345812,65,-1.115027
0,{'Car Name.': 'Toyota Altis automatic for sale...,Toyota Altis automatic for sale Toyota Corroll...,0.334184,69,-4.434132
4,"{'Car Name.': 'Corolla Altis 1.6 automatic', '...",Corolla Altis 1.6 automatic Toyota Corrolla Al...,0.348494,262,-4.627136


In [ ]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [ ]:
top_3_RAG

,Documents,Metadatas
3,HONDA CITY 2001 Auto Honda City IDSI 2001 9500...,"{'Car Name.': 'HONDA CITY 2001 Auto', 'Make': ..."
0,Toyota Altis automatic for sale Toyota Corroll...,{'Car Name.': 'Toyota Altis automatic for sale...
4,Corolla Altis 1.6 automatic Toyota Corrolla Al...,"{'Car Name.': 'Corolla Altis 1.6 automatic', '..."


In [ ]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the car domain who can effectively answer user queries about car model, features and other details."},
                {"role": "user", "content": f"""You are a helpful assistant in the car domain who can effectively answer user queries about car model, features and other details.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of car details in the dataframe '{top_3_RAG}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

                                                The column 'car_details' inside this dataframe contains the actual text from the cars csv file and the column 'metadata' contains the car make and car name.

                                                Use the car_details in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant car names and car make.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the car names and car makes.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific features or car names.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the car details, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the car names.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [ ]:
# Generate the response

response = generate_response(query, top_3_RAG)

In [ ]:
# Print the response

print("\n".join(response))

Based on the information from the given dataframe, the petrol cars with automatic transmission and manufactured after 2001 are:

1. Car Name: HONDA CITY 2001 Auto
   Make: Honda

2. Car Name: Toyota Altis automatic for sale
   Make: Toyota

3. Car Name: Corolla Altis 1.6 automatic
   Make: Toyota

These are the relevant petrol cars with automatic transmission that were manufactured after 2001 which are available based on the provided data.
